In [1]:
from feat import Detector # py-feat
import imageio.v2 as imageio # to create video from frames
import io

In [ ]:
face_model = 'img2pose'
landmark_model='mobilenet' 
facepose_model='img2pose-c'


detector = Detector(face_model=face_model, 
                    landmark_model=landmark_model, 
                    au_model='xgb', 
                    emotion_model='resmasknet', 
                    facepose_model=facepose_model,
                    identity_model='facenet', 
                    device='cuda', 
                    n_jobs=1, 
                    verbose=True,
                    )

In [3]:
video_path = "F:\\RAVDESS\\Actor_11\\01-01-07-01-02-01-11.mp4"
print(video_path)
output_filename = f'F:\\tracked_overlay\\testing\\720_1280\\{video_path[-24:-4]}_{face_model}_{landmark_model}_{facepose_model}_testing.mp4'
print(output_filename)

F:\RAVDESS\Actor_11\01-01-07-01-02-01-11.mp4
F:\tracked_overlay\testing\720_1280\01-01-07-01-02-01-11_img2pose_mobilenet_img2pose-c_testing.mp4


In [ ]:
# video_path = "F:\\CREMA-D\\1004_DFA_ANG_XX.mp4"
# output_filename = "F:\\CREMA-D\\1004_DFA_ANG_XX_overlay.mp4"

In [4]:
print("running py-feat detector")
# run detector
video_prediction = detector.detect_video(video_path, 
                                         skip_frames=None, 
                                         output_size=(720, 1280), 
                                         batch_size=5, 
                                         num_workers=0, 
                                         pin_memory=False, 
                                         face_detection_threshold=0.9, 
                                         face_identity_threshold=0.8
                                         )


running py-feat detector


  0%|          | 0/21 [00:00<?, ?it/s]INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:detecting faces...
INFO:root:detecting landmarks...
INFO:root:RESCALING WARNING: image_operations.extract_face_from_bbox() is rescaling cropped img with shape torch.Size([3, 582, 582]) to 224
INFO:root:RESCALING WARNING: image_operations.extract_face_from_bbox() is rescaling cropped img with shape torch.Size([3, 582, 582]) to 224
INFO:root:RESCALING WARNING: image_operations.extract_face_from_bbox() is rescaling cropped img with shape torch.Size([3, 582, 582]) to 224
INFO:root:RESCALING WARNING: image_operations

In [ ]:

# Constants for video settings
VIDEO_WIDTH, VIDEO_HEIGHT = 1280, 720  # Desired resolutionFalse
DPI = 100  # Resolution in dots per inch

print("generating overlay for each frame")
# generate overlay for each frame
figs = video_prediction.plot_detections(faces='landmarks', 
                                          faceboxes=True, 
                                          muscles=False, 
                                          poses=True, 
                                          gazes=False, 
                                          add_titles=False, 
                                          au_barplot=False, 
                                          emotion_barplot=False, 
                                          plot_original_image=True
                                          );

# Create a video writer object, specifying the output file, codec, and framerate
writer = imageio.get_writer(output_filename, fps=30, codec='libx264', format='FFMPEG', macro_block_size=None)

print("generating video")
# Loop through each figure in the list, adjust size, save it to a buffer, and append it to the video
for fig in figs:
    # Set the figure size to correspond to the desired pixel dimensions
    fig.set_size_inches(VIDEO_WIDTH / DPI, VIDEO_HEIGHT / DPI)
    
    # Use a buffer to save the figure
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=DPI)
    buf.seek(0)
    
    # Read the image from the buffer and add it to the video
    image = imageio.imread(buf)
    writer.append_data(image)
    
    # Close the buffer
    buf.close()

# Close the writer to finalize the video
writer.close()

print(f'Video saved as {output_filename}')
